In [1]:
# Basic utilities
import re
import glob
import time
import sys
from datetime import date
from copy import deepcopy
from skimage.exposure import rescale_intensity
import matplotlib.pyplot as plt

# Data wrangling packages
import numpy as np
from scipy import stats
from skimage.filters import threshold_yen

# Cellpose - the current belle of the ball
from cellpose import models, io

# Grab utilities from the imaging directories dir
utilsDir = re.sub(r'Cell Segmentation', 'Imaging Utilities', str(sys.path[0]))

# Homebrew utilities for importing ims files & basic image manipulation
sys.path.append(utilsDir)
import ims_file_utils as ims
import image_processing_utils as impro
from skimage.segmentation import find_boundaries

from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import scipy.ndimage as ndimage
import matplotlib.pyplot as plt
from skimage import io
import glob
from skimage.filters import threshold_local
from skimage.filters import threshold_li
from skimage.color import rgb2gray
from math import sqrt
from skimage.morphology import disk, ball
from skimage.feature import blob_dog, blob_log, blob_doh
from skimage.filters.rank import enhance_contrast
from skimage.exposure import adjust_gamma
import pickle as pkl
import sys
from datetime import date
import os, psutil
import re
process = psutil.Process(os.getpid())
# Grab utilities from the imaging directories dir
utilsDir = re.sub(r'Registration', 'Imaging Utilities', str(sys.path[0]))
# Homebrew utilities for importing ims files & basic image manipulation
sys.path.append(utilsDir)
#import ims_file_utils as ims
#import image_processing_utils as impro
#import plotting_funcs as plot
from functools import reduce
from scipy.spatial.distance import cdist, pdist
from skimage.exposure import rescale_intensity
from skimage.segmentation import find_boundaries

# Use this function for local thresholding of images
def lthresh(imgstk):
    th = threshold_local(imgstk, 65, offset=0)
    imgstk = imgstk>= th
    return imgstk
# Use this function to threshold images
def thresh(imgstk):
    th = threshold_li(imgstk)
    imgstk = imgstk>= th
    return imgstk
def spotcall(img, min_sig, max_sig, nsig, th):
    blobs_log = blob_log(img, min_sigma=min_sig, max_sigma=max_sig, num_sigma=nsig, threshold=th)
    # Compute radii in the 3rd column.
    blobs_log[:, 2] = blobs_log[:, 2] * sqrt(2)
    color = 'lime'
    title = 'Laplacian of Gaussian'
    #plt.rcParams['figure.figsize'] = [20, 15]
    #fig, ax = plt.subplots(1, 1, sharex=True, sharey=True)
    #ax.set_title(title)
    #ax.imshow(img)
    #for blob in blobs_log:
    #    y, x, r = blob
    #    c = plt.Circle((x, y), r, color=color, linewidth=2, fill=False)
    #    ax.add_patch(c)
    #ax.set_axis_off()
    #plt.tight_layout()
    #plt.show()
    return blobs_log


# Basic utilities
from datetime import date
from copy import deepcopy
# Data wrangling packages
from scipy import stats
from skimage.filters import threshold_yen
# Cellpose - the current belle of the ball
from cellpose import models, io
import pandas as pd
from math import sqrt

def numpyToList(points):
    # Takes a tuple of per-axis coordinates and converts it to a list of tuples
    outList = list()
    for i in range(len(points[0])):
        point = list()
        for axis in points:
            point.append(axis[i])
        outList.append(tuple(point))
    return(outList)

def fociDetection(image, masks_nuc, name, outFile):
    # Use laplacian of gaussian to detect blobs
    spots = spotcall(image,min_sig=0.05, max_sig=1,nsig=3,th=0.5) # Decrease threshold if spots are not identified and min sigma if spots are smaller
    spots = np.unique(spots, axis=0)
    print("Num " + name + " foci detected:", len(spots))

    # save spot coordinates
    with open(outFile + '_' + name + '_' + str(len(spots)) + '.pkl', 'wb') as filehandler:
        pkl.dump(spots, filehandler)
        
    return(spots)

In [2]:
runlist = ['A3_5','A3_6','A3_7','A3_8','A3_9',
          'A4_1', 'A4_2','A4_3','A4_4','A4_5','A4_6','A4_8',
          'A5_1', 'A5_2','A5_3','A5_4','A5_5','A5_6','A5_8']#'A1_1', 'A1_2','A1_3','A1_4','A1_5','A1_6','A1_8','A1_9','A2_1', 'A2_2','A2_3','A2_4','A2_5','A2_6','A2_8','A3_1', 'A3_2','A3_3','A3_4',
for runName in runlist:
    start_time = time.time()
    opdir = '/mnt/disks/external/jg4159/20221215_DDR364xDrugs/' # Path to output
    ip = opdir + "RegisteredImages_Ab_" + runName + ".pickle"
    mk = opdir+ runName + "_segmentation_ref_mem_2D_RAD51&yH2AX&RPA2_micronuclei_corrected.pickle"
    print(runName)

    #Loading ims files for different cycles
    lof = glob.glob(mk)
    filehandler = open(lof[0], 'rb')
    masks_mem, masks_nuc, ref_mem = pkl.load(filehandler)
    print(len(ref_mem))
    filehandler.close()

    filehandler = open(ip, 'rb')
    img_stack = pkl.load(filehandler)
    filehandler.close()

    for img in img_stack[1:]:
        for i in range(img.shape[0]):
            img[i] *= 65535
            img[i] = img[i].astype(np.uint16)
            m = np.median(img[i][img[i] > 0])
            #print(np.percentile(img[i], (97.5, 99.8)))

    rad_p2 = 118
    rad_p98 = 130
    rad = rescale_intensity(img_stack[0][1].astype('float64'), (rad_p2, rad_p98), (0, 255))

    yH2AX_p2 = 105
    yH2AX_p98 = 120
    yH2AX = rescale_intensity(img_stack[0][2].astype('float64'), (yH2AX_p2, yH2AX_p98), (0, 255))

    RPA2_p2 = 112
    RPA2_p98 = 125
    RPA2 = rescale_intensity(img_stack[0][3].astype('float64'), (RPA2_p2, RPA2_p98), (0, 255))

    # Populate both dictionaries
    for key in ref_mem.keys(): 
        mem_pxls = ref_mem[key]['Membrane Pixels 2D']
        nuc_pxls = ref_mem[key]['Nuclei Pixels 2D']
        cyto_pxls = (np.array(mem_pxls[0][masks_nuc[mem_pxls]==0]), np.array(mem_pxls[1][masks_nuc[mem_pxls]==0]))
        ref_mem[key]["Cytoplasm Pixels 2D"] = cyto_pxls
        try:#np.max(mem_pxls) > :
            #ref_mem[key]['out of bound'] = True
        #else:
            #ref_mem[key]['out of bound'] = False
            # Get cyc Stats
            sum_EPCAM_mem = np.sum(img_stack[0][0][mem_pxls])
            sum_yH2AX_nuc = np.sum(img_stack[0][2][nuc_pxls])
            sum_Ki67_nuc = np.sum(img_stack[1][0][nuc_pxls])
            sum_Ki67_mem = np.sum(img_stack[1][0][mem_pxls])
            sum_cycB1_nuc = np.sum(img_stack[1][1][nuc_pxls])
            sum_cycB1_mem = np.sum(img_stack[1][1][mem_pxls])
            sum_cycB1_cyto = np.sum(img_stack[1][1][cyto_pxls])
            sum_phoH1_nuc = np.sum(img_stack[1][2][nuc_pxls])
            sum_phoH1_mem = np.sum(img_stack[1][2][mem_pxls])
            sum_cycA_nuc = np.sum(img_stack[1][3][nuc_pxls])
            sum_cycA_mem = np.sum(img_stack[1][3][mem_pxls])
            sum_lamin_nuc = np.sum(img_stack[2][0][nuc_pxls])
            sum_lamin_mem = np.sum(img_stack[2][0][mem_pxls])
            sum_cPARP_nuc = np.sum(img_stack[2][1][nuc_pxls])
            sum_cPARP_mem = np.sum(img_stack[2][1][mem_pxls]) 
            sum_p21_nuc = np.sum(img_stack[2][2][nuc_pxls])
            sum_p21_mem = np.sum(img_stack[2][2][mem_pxls])
            sum_p21_cyto = np.sum(img_stack[2][2][cyto_pxls])
            sum_p53_nuc = np.sum(img_stack[2][3][nuc_pxls])
            sum_p53_mem = np.sum(img_stack[2][3][mem_pxls])
            sum_p53_cyto = np.sum(img_stack[2][3][cyto_pxls])
            sum_Tub_nuc = np.sum(img_stack[3][0][nuc_pxls])
            sum_Tub_mem = np.sum(img_stack[3][0][mem_pxls])
            sum_STAT1_nuc = np.sum(img_stack[3][1][nuc_pxls])
            sum_STAT1_mem = np.sum(img_stack[3][1][mem_pxls])
            sum_STAT1_cyto = np.sum(img_stack[3][1][cyto_pxls])
            sum_IRF3_nuc = np.sum(img_stack[3][2][nuc_pxls])
            sum_IRF3_mem = np.sum(img_stack[3][2][mem_pxls])
            sum_IRF3_cyto = np.sum(img_stack[3][2][cyto_pxls])
            sum_p65_nuc = np.sum(img_stack[3][3][nuc_pxls])
            sum_p65_mem = np.sum(img_stack[3][3][mem_pxls])
            sum_p65_cyto = np.sum(img_stack[3][3][cyto_pxls])

            # Get GFP Stats
            ref_mem[key]['Avg EPCAM under Membrane'] = sum_EPCAM_mem
            ref_mem[key]['Avg yH2AX under Nuclei'] = sum_yH2AX_nuc
            ref_mem[key]['Avg Ki67 under Membrane'] = sum_Ki67_mem
            ref_mem[key]['Avg Ki67 under Nuclei'] = sum_Ki67_nuc
            ref_mem[key]['Avg cycB1 under Membrane'] = sum_cycB1_mem
            ref_mem[key]['Avg cycB1 under Nuclei'] = sum_cycB1_nuc
            ref_mem[key]['Avg cycB1 under Cytoplasm'] = sum_cycB1_cyto
            ref_mem[key]['Avg phoH3 under Membrane'] = sum_phoH1_mem
            ref_mem[key]['Avg phoH3 under Nuclei'] = sum_phoH1_nuc
            ref_mem[key]['Avg cycA under Membrane'] = sum_cycA_mem
            ref_mem[key]['Avg cycA under Nuclei'] = sum_cycA_nuc
            ref_mem[key]['Avg lamin under Membrane'] = sum_lamin_mem
            ref_mem[key]['Avg lamin under Nuclei'] = sum_lamin_nuc
            ref_mem[key]['Avg cPARP under Membrane'] = sum_cPARP_mem
            ref_mem[key]['Avg cPARP under Nuclei'] = sum_cPARP_nuc
            ref_mem[key]['Avg p21 under Membrane'] = sum_p21_mem
            ref_mem[key]['Avg p21 under Nuclei'] = sum_p21_nuc
            ref_mem[key]['Avg p21 under Cytoplasm'] = sum_p21_cyto
            ref_mem[key]['Avg p53 under Membrane'] = sum_p53_mem
            ref_mem[key]['Avg p53 under Nuclei'] = sum_p53_nuc
            ref_mem[key]['Avg p53 under Cytoplasm'] = sum_p53_cyto
            ref_mem[key]['Avg Tub under Membrane'] = sum_Tub_mem
            ref_mem[key]['Avg Tub under Nuclei'] = sum_Tub_nuc
            ref_mem[key]['Avg STAT1 under Membrane'] = sum_STAT1_mem
            ref_mem[key]['Avg STAT1 under Nuclei'] = sum_STAT1_nuc
            ref_mem[key]['Avg STAT1 under Cytoplasm'] = sum_STAT1_cyto
            ref_mem[key]['Avg IRF3 under Membrane'] = sum_IRF3_mem
            ref_mem[key]['Avg IRF3 under Nuclei'] = sum_IRF3_nuc
            ref_mem[key]['Avg IRF3 under Cytoplasm'] = sum_IRF3_cyto
            ref_mem[key]['Avg p65 under Membrane'] = sum_p65_mem
            ref_mem[key]['Avg p65 under Nuclei'] = sum_p65_nuc
            ref_mem[key]['Avg p65 under Cytoplasm'] = sum_p65_cyto
        except IndexError:
            ref_mem[key]['out of bound'] = True
            #print(key)
            ref_mem[key]['Avg EPCAM under Membrane'] = 100*len(ref_mem[key]['Membrane Pixels'])
            ref_mem[key]['Avg yH2AX under Nuclei'] = 100*len(ref_mem[key]['Nuclei Pixels']) 
            ref_mem[key]['Avg Ki67 under Membrane'] = 100*len(ref_mem[key]['Membrane Pixels'])
            ref_mem[key]['Avg Ki67 under Nuclei'] = 100*len(ref_mem[key]['Nuclei Pixels']) 
            ref_mem[key]['Avg cycB1 under Membrane'] = 100*len(ref_mem[key]['Membrane Pixels'])
            ref_mem[key]['Avg cycB1 under Nuclei'] = 100*len(ref_mem[key]['Nuclei Pixels'])
            ref_mem[key]['Avg cycB1 under Cytoplasm'] = 100*len(ref_mem[key]['Nuclei Pixels'])
            ref_mem[key]['Avg phoH3 under Membrane'] = 100*len(ref_mem[key]['Membrane Pixels'])
            ref_mem[key]['Avg phoH3 under Nuclei'] = 100*len(ref_mem[key]['Nuclei Pixels']) 
            ref_mem[key]['Avg cycA under Membrane'] = 100*len(ref_mem[key]['Membrane Pixels'])
            ref_mem[key]['Avg cycA under Nuclei'] = 100*len(ref_mem[key]['Nuclei Pixels']) 
            ref_mem[key]['Avg lamin under Membrane'] = 100*len(ref_mem[key]['Membrane Pixels'])
            ref_mem[key]['Avg lamin under Nuclei'] = 100*len(ref_mem[key]['Nuclei Pixels']) 
            ref_mem[key]['Avg cPARP under Membrane'] = 100*len(ref_mem[key]['Membrane Pixels'])
            ref_mem[key]['Avg cPARP under Nuclei'] = 100*len(ref_mem[key]['Nuclei Pixels']) 
            ref_mem[key]['Avg p21 under Membrane'] = 100*len(ref_mem[key]['Membrane Pixels'])
            ref_mem[key]['Avg p21 under Nuclei'] = 100*len(ref_mem[key]['Nuclei Pixels'])
            ref_mem[key]['Avg p21 under Cytoplasm'] = 100*len(ref_mem[key]['Nuclei Pixels'])
            ref_mem[key]['Avg p53 under Membrane'] = 100*len(ref_mem[key]['Membrane Pixels'])
            ref_mem[key]['Avg p53 under Nuclei'] = 100*len(ref_mem[key]['Nuclei Pixels']) 
            ref_mem[key]['Avg p53 under Cytoplasm'] = 100*len(ref_mem[key]['Nuclei Pixels'])
            ref_mem[key]['Avg Tub under Membrane'] = 100*len(ref_mem[key]['Membrane Pixels'])
            ref_mem[key]['Avg Tub under Nuclei'] = 100*len(ref_mem[key]['Nuclei Pixels']) 
            ref_mem[key]['Avg STAT1 under Membrane'] = 100*len(ref_mem[key]['Membrane Pixels'])
            ref_mem[key]['Avg STAT1 under Nuclei'] = 100*len(ref_mem[key]['Nuclei Pixels']) 
            ref_mem[key]['Avg STAT1 under Cytoplasm'] = 100*len(ref_mem[key]['Nuclei Pixels']) 
            ref_mem[key]['Avg IRF3 under Membrane'] = 100*len(ref_mem[key]['Membrane Pixels'])
            ref_mem[key]['Avg IRF3 under Nuclei'] = 100*len(ref_mem[key]['Nuclei Pixels']) 
            ref_mem[key]['Avg IRF3 under Cytoplasm'] = 100*len(ref_mem[key]['Nuclei Pixels']) 
            ref_mem[key]['Avg p65 under Membrane'] = 100*len(ref_mem[key]['Membrane Pixels']) 
            ref_mem[key]['Avg p65 under Nuclei'] = 100*len(ref_mem[key]['Nuclei Pixels']) 
            ref_mem[key]['Avg p65 under Cytoplasm'] = 100*len(ref_mem[key]['Nuclei Pixels']) 
        else:
            ref_mem[key]['out of bound'] = False
        
    filename = opdir + runName + '_all_spots_'
    rad51_spots = fociDetection(rad, masks_nuc, 'RAD51', filename + str(rad_p2) + '_' + str(rad_p98))
    yh2ax_spots = fociDetection(yH2AX, masks_nuc, 'yH2AX', filename + str(yH2AX_p2) + '_' + str(yH2AX_p98))
    rpa2_spots = fociDetection(RPA2, masks_nuc, 'RPA2', filename + str(RPA2_p2) + '_' + str(RPA2_p98))

    # Strip the z axis off the detected foci coords and convert to integers
    detectedFoci = [rad51_spots[:,0:2].astype('uint16'),
                   yh2ax_spots[:,0:2].astype('uint16'), rpa2_spots[:,0:2].astype('uint16')]
    names = ('RAD51_spots_118', 'yH2AX_spots_105', 'RPA2_spots_112')#, 'RPA2 spots'

    # Create a dictionary where every key is a foci coordinate, and the value is the protein
    spotMatch = {}
    for i, spots in enumerate(detectedFoci):
        for spot in spots:
            if spotMatch.get((spot[0], spot[1])):
                spotMatch[(spot[0], spot[1])].append(names[i])
            else:
                spotMatch[(spot[0], spot[1])] = [names[i]]

    # For each nuclei, see which spots in the membrane are indexed in the key 
    for i, cellID in enumerate(ref_mem):
        for name in names:
            ref_mem[cellID][name] = 0

        # For each Nuclei Coordinate look for a spot
        nucCoord = numpyToList(ref_mem[cellID]['Nuclei Pixels 2D'])

        for coord in nucCoord:
            foci = spotMatch.get(coord)
            if foci:
                for focus in foci:
                    ref_mem[cellID][focus] += 1

    for i, mem_id in enumerate(ref_mem):
        #print(mem_id)
        mem_pxls = ref_mem[mem_id]["Membrane Pixels 2D"]
        # Get the most common membrane mask value for that location
        nucs = masks_nuc[mem_pxls]
        #cyto = masks_nuc[mem_pxls]
        if sum(nucs)>0: #nuc present
            nucs = nucs[nucs!=0] #non-zero nuc_ids under the membrane mask
            
            #nuc_id = stats.mode(nucs)[0][0] # pick the mode as the nuc_id
            nuc_id = []
            for j in np.unique(nucs):
                #print(j)
                nuc_id.append(j)
            nuc_size = [len(np.where(nucs==x)[0]) for x in nuc_id]
            #nuc_id_sorted = sorted(nuc_id, reverse=True, key=lambda x: nuc_size[nuc_id.index(x)])
            #nuc_pxls = [np.where(nucs==x)[0] for x in nuc_id_sorted]
            #nuc_pxls_2d = [np.where(masks_nuc==x) for x in nuc_id_sorted]

            #ref_mem[mem_id]["nuc_id_sorted"] =  nuc_id_sorted
            #ref_mem[mem_id]["nuc_pxls_sorted"] = nuc_pxls
            ref_mem[mem_id]["large_nuc"] = sum([x>=100 for x in nuc_size])
            #ref_mem[mem_id]["nuc_pxls_2d"] = nuc_pxls_2d
            #ref_mem[mem_id]["mem_pxls_2d"] = np.where(masks_mem==mem_id)

    #Produce output pickle
    with open(opdir + runName + "_ref_mem_2D_RAD51&yH2AX&RPA2_newfoci" + '.pickle', 'wb') as filehandler:
        pkl.dump((ref_mem), filehandler)

    data = []
    for key in ref_mem.keys(): # use keys in the saved dict
        if True:
            data.append([key, 
                ref_mem[key]['out of bound'],
                ref_mem[key]['RAD51 spots'],
                ref_mem[key]['yH2AX spots'],
                ref_mem[key]['RAD51_spots_118'],
                ref_mem[key]['yH2AX_spots_105'],
                ref_mem[key]['RPA2_spots_112'],
                ref_mem[key]['Avg EPCAM under Membrane'],
                ref_mem[key]['Avg yH2AX under Nuclei'] ,
                ref_mem[key]['Avg Ki67 under Membrane'] ,
                ref_mem[key]['Avg Ki67 under Nuclei'] ,
                ref_mem[key]['Avg cycB1 under Membrane'],
                ref_mem[key]['Avg cycB1 under Nuclei'],
                ref_mem[key]['Avg cycB1 under Cytoplasm'],
                ref_mem[key]['Avg phoH3 under Membrane'],
                ref_mem[key]['Avg phoH3 under Nuclei'] ,
                ref_mem[key]['Avg cycA under Membrane'] ,
                ref_mem[key]['Avg cycA under Nuclei'] ,
                ref_mem[key]['Avg lamin under Membrane'],
                ref_mem[key]['Avg lamin under Nuclei'] ,
                ref_mem[key]['Avg cPARP under Membrane'],
                ref_mem[key]['Avg cPARP under Nuclei'],
                ref_mem[key]['Avg p21 under Membrane'],
                ref_mem[key]['Avg p21 under Nuclei'] ,
                ref_mem[key]['Avg p21 under Cytoplasm'] ,
                ref_mem[key]['Avg p53 under Membrane'],
                ref_mem[key]['Avg p53 under Nuclei'] ,
                ref_mem[key]['Avg p53 under Cytoplasm'] ,
                ref_mem[key]['Avg Tub under Membrane'],
                ref_mem[key]['Avg Tub under Nuclei'],
                ref_mem[key]['Avg STAT1 under Membrane'],
                ref_mem[key]['Avg STAT1 under Nuclei'] ,
                ref_mem[key]['Avg STAT1 under Cytoplasm'] ,
                ref_mem[key]['Avg IRF3 under Membrane'],
                ref_mem[key]['Avg IRF3 under Nuclei'],
                ref_mem[key]['Avg IRF3 under Cytoplasm'],
                ref_mem[key]['Avg p65 under Membrane'],
                ref_mem[key]['Avg p65 under Nuclei'],
                ref_mem[key]['Avg p65 under Cytoplasm'],
                ref_mem[key]['nuc_present'],
                ref_mem[key]['large_nuc'],
                ref_mem[key]['micronuclei'],
                len(ref_mem[key]['Nuclei Pixels']), 
                len(ref_mem[key]['Membrane Pixels']),
                len(ref_mem[key]['Cytoplasm Pixels 2D'][0])
                        ])

    df2 = pd.DataFrame(data, columns=['cell_id',
                                      "out_of_bound",
                                'RAD51_foci',
                                'yH2AX_foci',
                                'RAD51_foci_118',
                                'yH2AX_foci_105',   
                                'RPA2_foci_112',
                                'sum_EPCAM_mem',
                                'sum_yH2AX_nuc',
                                'sum_Ki67_mem',
                                'sum_Ki67_nuc',
                                'sum_cycB1_mem',
                                'sum_cycB1_nuc',
                                'sum_cycB1_cyto',
                                'sum_phoH1_mem',
                                'sum_phoH1_nuc',
                                'sum_cycA_mem',
                                'sum_cycA_nuc',
                                'sum_lamin_mem',
                                'sum_lamin_nuc',
                                'sum_cPARP_mem',
                                'sum_cPARP_nuc',
                                'sum_p21_mem',
                                'sum_p21_nuc',
                                'sum_p21_cyto',
                                'sum_p53_mem',
                                'sum_p53_nuc',
                                'sum_p53_cyto',
                                'sum_Tub_mem',
                                'sum_Tub_nuc',
                                'sum_STAT1_mem',
                                'sum_STAT1_nuc',
                                'sum_STAT1_cyto',
                                'sum_IRF3_mem',
                                'sum_IRF3_nuc',
                                'sum_IRF3_cyto',
                                'sum_p65_mem',
                                'sum_p65_nuc',
                                'sum_p65_cyto',
                                      'nuc_present',
                                      "large_nuclei",
                                      "micronuclei_spots",
                                      "nuclei_size", 
                                      "mem_size","cyto_size"
                                     ])

    df2 = df2.sort_values(by="cell_id")
    df2.to_csv(opdir + runName+"_RAD51&yH2AX&RPA2_newfoci" + str(date.today()) + '.csv', index=False)
    print("--- %s seconds ---" % (time.time() - start_time))


A3_5
28279
Num RAD51 foci detected: 164192


/mnt/disks/external/jg4159/.conda/envs/cellpose/lib/python3.7/site-packages/skimage/feature/blob.py:125: RuntimeWarning: invalid value encountered in double_scalars
  r1 = blob1[-1] / blob2[-1]
/mnt/disks/external/jg4159/.conda/envs/cellpose/lib/python3.7/site-packages/skimage/feature/blob.py:126: RuntimeWarning: divide by zero encountered in true_divide
  pos1 = blob1[:ndim] / (max_sigma * root_ndim)
/mnt/disks/external/jg4159/.conda/envs/cellpose/lib/python3.7/site-packages/skimage/feature/blob.py:127: RuntimeWarning: divide by zero encountered in true_divide
  pos2 = blob2[:ndim] / (max_sigma * root_ndim)
/mnt/disks/external/jg4159/.conda/envs/cellpose/lib/python3.7/site-packages/skimage/feature/blob.py:129: RuntimeWarning: invalid value encountered in subtract
  d = np.sqrt(np.sum((pos2 - pos1)**2))


Num yH2AX foci detected: 286070
Num RPA2 foci detected: 171104
--- 78.9158034324646 seconds ---
A3_6
17540
Num RAD51 foci detected: 108573
Num yH2AX foci detected: 176485
Num RPA2 foci detected: 186849
--- 114.18708372116089 seconds ---
A3_7
16728
Num RAD51 foci detected: 113239
Num yH2AX foci detected: 195250
Num RPA2 foci detected: 233095
--- 153.92729139328003 seconds ---
A3_8
22384
Num RAD51 foci detected: 155492
Num yH2AX foci detected: 269393
Num RPA2 foci detected: 223311
--- 158.68643927574158 seconds ---
A3_9
20266
Num RAD51 foci detected: 166255
Num yH2AX foci detected: 244761
Num RPA2 foci detected: 253745
--- 162.13278818130493 seconds ---
A4_1
31227
Num RAD51 foci detected: 64780
Num yH2AX foci detected: 548400
Num RPA2 foci detected: 177324
--- 175.29338693618774 seconds ---
A4_2
30201
Num RAD51 foci detected: 73871
Num yH2AX foci detected: 527876
Num RPA2 foci detected: 252675
--- 176.22144484519958 seconds ---
A4_3
28766
Num RAD51 foci detected: 60952
Num yH2AX foci det